In [1]:
import csv

# read demand data
f = open("module2_quiz_P2a.csv")
csvfile = csv.DictReader(f)
headers = csvfile.fieldnames

table = []
for row in csvfile:
    table.append(row)
    
f.close()

Groups = []
Demand = {}
Months = range(1,6)
for row in table:
    Groups.append(row['Demand (tonne)'])
    for m in Months:
        Demand[(row['Demand (tonne)'], m)] = float(row[str("Month "+str(m))])

# read cost data
f = open("module2_quiz_P2b.csv")
csvfile = csv.DictReader(f)
headers = csvfile.fieldnames

table = []
for row in csvfile:
    table.append(row)
    
f.close()
                
Cost = {}
for row in table:
    for m in Months:
        Cost[(row['Production cost ($/tonne)'], m)] = float(row[str("Month "+str(m))])

In [2]:
from docplex.mp.model import Model
mdl = Model()

In [3]:
# Decision variables
x = mdl.continuous_var_matrix(Groups, Months, lb=0, name='production')
s = mdl.continuous_var_matrix(Groups, Months, lb=0, name='storage')

# Objective function
total_cost = mdl.sum(Cost[grp, m] * x[grp, m] for grp in Groups for m in Months) + \
             50 * mdl.sum(s[grp, m] for grp in Groups for m in Months)
mdl.minimize(total_cost)

# Constraints
for m in Months:
    # Processing capacity constraint
    mdl.add_constraint(mdl.sum(x[grp, m] for grp in Groups) <= 175, f'processing_capacity_{m}')
    
    # Storage capacity constraint
    mdl.add_constraint(mdl.sum(s[grp, m] for grp in Groups) <= 50, f'storage_capacity_{m}')
    
    for grp in Groups:
        if m == 1:
            # Updated initial inventory constraints
            if grp == 'Medium':
                available_storage = 5.3
            elif grp == 'Large':
                available_storage = 2.3
            elif grp == 'Small':
                available_storage = 3.7
            else:
                available_storage = 0
        else:
            available_storage = 0.96 * s[grp, m-1]
        
        # Demand constraint
        mdl.add_constraint(x[grp, m] + available_storage - s[grp, m] == Demand[grp, m], f'demand_{grp}_{m}')



In [4]:
# solve
mdl.solve()
mdl.get_solve_details()

docplex.mp.SolveDetails(time=0,status='optimal')

In [5]:
mdl.print_solution()

objective: 253176.004
status: OPTIMAL_SOLUTION(2)
  production_Large_1=28.700
  production_Large_2=25.000
  production_Large_3=45.000
  production_Large_4=59.000
  production_Large_5=44.000
  production_Medium_1=44.700
  production_Medium_2=41.000
  production_Medium_3=70.000
  production_Medium_4=80.000
  production_Medium_5=60.000
  production_Small_1=38.300
  production_Small_2=63.854
  production_Small_3=60.000
  production_Small_4=36.000
  production_Small_5=56.000
  storage_Small_2=33.854
  storage_Small_3=37.500
